In [ ]:
import pyhepmc
import vector
import numpy as np

## Read jet

In [ ]:
with pyhepmc.open("../data/events.hepmc3/events.hepmc3") as jets:
    jet = jets.read()
print(f"Total particles in jet: {len(jet.particles)}")


In [ ]:
# Count final state particles here, so arrays can be setup with the correct size
fsparticles = []
for iparticle, particle in enumerate(jet.particles):
    # Particle status 1 = Undecayed physical particle
    if particle.status == 1:
        fsparticles.append(iparticle)
print(f"Final state particles: {len(fsparticles)}")
nparticles = len(fsparticles)

## Construct jet particles as a numpy array of momentum vector objects

In [ ]:
# Pick only final state particles and construct arrays of properties
myPseudoJet = []
px = np.zeros(nparticles)
py = np.zeros(nparticles)
pz = np.zeros(nparticles)
E = np.zeros(nparticles)
for iparticle, fsparticle in enumerate(fsparticles):
    myPseudoJet.append(jet.particles[fsparticle])
    px[iparticle] = jet.particles[fsparticle].momentum.px
    py[iparticle] = jet.particles[fsparticle].momentum.py
    pz[iparticle] = jet.particles[fsparticle].momentum.pz
    E[iparticle] = jet.particles[fsparticle].momentum.e

In [ ]:
particles = vector.array({"px": px, "py": py, "pz": pz, "e": E})

In [ ]:
particles[0]

In [ ]:
print(particles[0].pt, particles[0].rho, particles[0].tau, particles[0].mass)

In [ ]:
# import pprint
# pprint.pprint([ str(prop) for prop in dir(pj[0]) if not prop.startswith("_") ])

In [ ]:
print(particles[1].y, particles[1].phi, particles[1].pt, particles[1].rapidity)

## Make some simple plots of the original particles

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
phi = np.zeros(len(pj))
y = np.zeros(len(pj))
pt = np.zeros(len(pj))
for iparticle, particle in enumerate(particles):
    y[iparticle] = particle.y
    phi[iparticle] = particle.phi
    pt[iparticle] = particle.pt

In [ ]:
# for p in zip(y, phi, pt):
#     print(p)

In [ ]:
levels = np.linspace(pt.min(), pt.max(), 7)
print(levels)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y, phi, 'o')
ax.tricontourf(y, phi, pt)
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter(y, phi, pt)
plt.show()

## Define distance measures

In [ ]:
def delta2(pj1, pj2):
    '''Return the geometric distance between two pseudojets'''
    return (pj1.y - pj2.y)**2 + (pj1.phi - pj2.phi)**2

In [ ]:
delta2(particles[0], particles[1])

In [ ]:
def akt_distance(pj1, pj2, r2=0.16):
    '''Return the antikt metric between two pseudojets (note the ^-2 in the momentum term)'''
    min_momentum = min(pj1.pt**-2, pj2.pt**-2)
    return min_momentum * delta2(pj1, pj2) / r2

In [ ]:
akt_distance(particles[0], particles[1])

In [ ]:
def beamline_distance(pj):
    return pj.pt**-2

In [ ]:
beamline_distance(particles[0])

In [ ]:
for n, p in enumerate(particles[1:], start=1):
    print(f"Distance to particle {n}:", akt_distance(particles[0], p))